In [41]:
import numpy as np
import pandas as pd

In [42]:
import spacy

In [43]:
from experiments import CHAR_VOCAB_PATH
from utils import generate_vocab_mappings, load_sentences

In [44]:
itos, stoi = generate_vocab_mappings(CHAR_VOCAB_PATH)

In [45]:
sentences_w_number = load_sentences("input_sentences/refactored_sentences.txt")

In [46]:
nlp = spacy.load('de_core_news_sm')

### Generating german dictionary 

In [47]:
def generate_german_dict(path):
    # TODO: path is "vocabularies/de_dict.csv"
    df = pd.read_csv(path)
    df = df[['lemma','genus','akkusativ singular','akkusativ singular 1','dativ singular',
                'dativ singular 1','genitiv singular','genitiv singular 1',
             'nominativ plural','akkusativ plural','akkusativ plural 1','dativ plural',
                'dativ plural 1','genitiv plural','genitiv plural 1']]
    df = df.rename(columns={
                                 'lemma':'nom sg',
                                 'akkusativ singular':'acc sg',
                                 'akkusativ singular 1':'acc1 sg',
                                 'dativ singular':'dat sg',
                                 'dativ singular 1':'dat1 sg',
                                 'genitiv singular':'gen sg',
                                 'genitiv singular 1':'gen1 sg',
                                 'nominativ plural':'nom pl',
                                 'akkusativ plural':'acc pl',
                                 'akkusativ plural 1':'acc1 pl',
                                 'dativ plural':'dat pl',
                                 'dativ plural 1':'dat1 pl',
                                 'genitiv plural':'gen pl',
                                 'genitiv plural 1':'gen1 pl'
                                 })
    
    df = df.replace(np.nan, '', regex=True)
    df['acc sg'] = df['acc sg'] + df['acc1 sg']
    df['dat sg'] = df['dat sg'] + df['dat1 sg']
    df['gen sg'] = df['gen sg'] + df['gen1 sg']
    df['acc pl'] = df['acc pl'] + df['acc1 pl']
    df['dat pl'] = df['dat pl'] + df['dat1 pl']
    df['gen pl'] = df['gen pl'] + df['gen1 pl']
    df = df.drop(['acc1 sg','dat1 sg','gen1 sg','acc1 pl','dat1 pl','gen1 pl'], axis=1)
    #duplicates = df.duplicated(subset=None, keep='first')
    print(df.shape)
    german_dict = df.set_index('nom sg').T.to_dict('list')
    return german_dict, df

In [48]:
german_dict, df = generate_german_dict("vocabularies/de_dict.csv")

(79203, 9)


/Users/charlotterochereau/anaconda3/envs/char_lm_fb/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: DataFrame columns are not unique, some columns will be omitted.


In [49]:
# testing german_dict
print(german_dict["Geschenk"])
print(german_dict["Mann"])
print(german_dict["Junge"])
print(german_dict["Journalist"])
print(german_dict["Präsident"])
print(german_dict["Mädchen"])
print(german_dict["Bett"])
print(german_dict["Held"])
print(german_dict["Arzt"])
print(german_dict["Tisch"])
print(german_dict["Platz"])

['n', 'Geschenk', 'Geschenk', 'Geschenkes', 'Geschenke', 'Geschenke', 'Geschenken', 'Geschenke']
['m', 'Mann', 'Mann', 'Mannes', '', 'Männer', 'Männern', 'Männer']
['m', 'Jungen', 'Jungen', 'Jungen', '', 'Jungen', 'Jungen', 'Jungen']
['m', 'Journalisten', 'Journalisten', 'Journalisten', 'Journalisten', 'Journalisten', 'Journalisten', 'Journalisten']
['m', 'Präsidenten', 'Präsidenten', 'Präsidenten', 'Präsidenten', 'Präsidenten', 'Präsidenten', 'Präsidenten']
['n', 'Mädchen', 'Mädchen', 'Mädchens', 'Mädchen', 'Mädchen', 'Mädchen', 'Mädchen']
['n', 'Bett', 'Bett', 'Betts', 'Betten', 'Betten', 'Betten', 'Betten']
['m', 'Helden', 'Helden', 'Helden', '', 'Helden', 'Helden', 'Helden']
['m', 'Arzt', 'Arzt', 'Arztes', 'Ärzte', 'Ärzte', 'Ärzten', 'Ärzte']
['m', 'Tisch', 'Tisch', 'Tischs', 'Tische', 'Tische', 'Tischen', 'Tische']
['m', 'Platz', 'Platz', 'Platzes', 'Plätze', 'Plätze', 'Plätzen', 'Plätze']


### Parsing using spacy

In [50]:
print(sentences_w_number)

Wir glauben, dass der Mann sg die Frau pl ein Geschenk sg gegeben hat. Wir wissen, dass der Bankier sg sein Kunde pl eine Finanzierung sg angeboten hat. Wir wissen, dass das Kindermädchen sg das Kind sg ein Buch sg gekauft hat. Wir vermuten, dass die Hausherrin sg sein Gast pl ein Wein sg angebietet hat. Wir denken, dass der Mann sg sein Freund pl eine Sehenswürdigkeit pl gezeigt hat. Wir bezweifeln, dass der Verdächtiger sg sein Anwalt sg die Wahrheit sg verheimlicht hat. Wir denken, dass der Arzt sg der Patient sg eine Behandlung sg verschieben hat. Wir erfahren, dass der Eigentümer sg ein Student sg seine Wohnung sg vermietet hat. Wir verdächtigen, dass der Spion sg ihr Feind sg das Geheimnis pl verratet hat. Wir denken, dass die Frau sg sein Partner sg eine Überraschung sg versprochen hat. Ihr weisst, dass die Mutter sg ihre Tochter sg ein Paket sg geschickt hat. Sie sagen, dass der Sportler sg seine Nachbarin sg sein Fahrrad sg verliehen hat. Wir erfahren, dass die Presse sg der Z

In [72]:
sentences_wo_number = sentences_w_number.replace('sg ','').replace('pl ','')

In [73]:
print(sentences_wo_number)

Wir glauben, dass der Mann die Frau ein Geschenk gegeben hat. Wir wissen, dass der Bankier sein Kunde eine Finanzierung angeboten hat. Wir wissen, dass das Kindermädchen das Kind ein Buch gekauft hat. Wir vermuten, dass die Hausherrin sein Gast ein Wein angebietet hat. Wir denken, dass der Mann sein Freund eine Sehenswürdigkeit gezeigt hat. Wir bezweifeln, dass der Verdächtiger sein Anwalt die Wahrheit verheimlicht hat. Wir denken, dass der Arzt der Patient eine Behandlung verschieben hat. Wir erfahren, dass der Eigentümer ein Student seine Wohnung vermietet hat. Wir verdächtigen, dass der Spion ihr Feind das Geheimnis verratet hat. Wir denken, dass die Frau sein Partner eine Überraschung versprochen hat. Ihr weisst, dass die Mutter ihre Tochter ein Paket geschickt hat. Sie sagen, dass der Sportler seine Nachbarin sein Fahrrad verliehen hat. Wir erfahren, dass die Presse der Zuhörer die Nachricht verkündet hat. Sie hören, dass der Senat die Abgeordnete eine Maßnahme gemeldet hat. Ich g

In [74]:
doc_w_number = nlp(sentences_w_number)
print("doc with number:",len(doc_w_number))
doc_wo_number = nlp(sentences_wo_number)
print("doc without number:",len(doc_wo_number))

doc with number: 304
doc without number: 247


### Sentence segmenter

In [75]:
list_sentences_w_number = []
for sentence in doc_w_number.sents:
    list_sentences_w_number.append(sentence.text)
print(list_sentences_w_number)

['Wir glauben, dass der Mann sg die Frau pl ein Geschenk sg gegeben hat.', 'Wir wissen, dass der Bankier sg sein Kunde pl eine Finanzierung sg angeboten hat.', 'Wir wissen, dass das Kindermädchen sg das Kind sg ein Buch sg gekauft hat.', 'Wir vermuten, dass die Hausherrin sg sein Gast pl ein Wein sg angebietet hat.', 'Wir denken, dass der Mann sg sein Freund pl eine Sehenswürdigkeit pl gezeigt hat.', 'Wir bezweifeln, dass der Verdächtiger sg sein Anwalt sg die Wahrheit sg verheimlicht hat.', 'Wir denken, dass der Arzt sg der Patient sg eine Behandlung sg verschieben hat.', 'Wir erfahren, dass der Eigentümer sg ein Student sg seine Wohnung sg vermietet hat.', 'Wir verdächtigen, dass der Spion sg ihr Feind sg das Geheimnis pl verratet hat.', 'Wir denken, dass die Frau sg sein Partner sg eine Überraschung sg versprochen hat.', 'Ihr weisst, dass die Mutter sg ihre Tochter sg ein Paket sg geschickt hat.', 'Sie sagen, dass der Sportler sg seine Nachbarin sg sein Fahrrad sg verliehen hat.', '

In [76]:
list_sentences_wo_number = []
for sentence in doc_wo_number.sents:
    list_sentences_wo_number.append(sentence.text)
print(list_sentences_wo_number)

['Wir glauben, dass der Mann die Frau ein Geschenk gegeben hat.', 'Wir wissen, dass der Bankier sein Kunde eine Finanzierung angeboten hat.', 'Wir wissen, dass das Kindermädchen das Kind ein Buch gekauft hat.', 'Wir vermuten, dass die Hausherrin sein Gast ein Wein angebietet hat.', 'Wir denken, dass der Mann sein Freund eine Sehenswürdigkeit gezeigt hat.', 'Wir bezweifeln, dass der Verdächtiger sein Anwalt die Wahrheit verheimlicht hat.', 'Wir denken, dass der Arzt der Patient eine Behandlung verschieben hat.', 'Wir erfahren, dass der Eigentümer ein Student seine Wohnung vermietet hat.', 'Wir verdächtigen, dass der Spion ihr Feind das Geheimnis verratet hat.', 'Wir denken, dass die Frau sein Partner eine Überraschung versprochen hat.', 'Ihr weisst, dass die Mutter ihre Tochter ein Paket geschickt hat.', 'Sie sagen, dass der Sportler seine Nachbarin sein Fahrrad verliehen hat.', 'Wir erfahren, dass die Presse der Zuhörer die Nachricht verkündet hat.', 'Sie hören, dass der Senat die Abge

In [77]:
assert len(list_sentences_w_number) == len(list_sentences_wo_number)
print("number of sentences:",len(list_sentences_w_number))
print("characters in sentences with number:", len(list_sentences_w_number[0]))
print("characters in sentences without number:", len(list_sentences_wo_number[0]))

number of sentences: 19
characters in sentences with number: 70
characters in sentences without number: 61


### Tokenizing using spacy

In [78]:
doc_list_w_number = []
for sentence in range(len(list_sentences_w_number)):
    doc_list_w_number.append(nlp(list_sentences_w_number[sentence]))

In [79]:
print(doc_list_w_number)

[Wir glauben, dass der Mann sg die Frau pl ein Geschenk sg gegeben hat., Wir wissen, dass der Bankier sg sein Kunde pl eine Finanzierung sg angeboten hat., Wir wissen, dass das Kindermädchen sg das Kind sg ein Buch sg gekauft hat., Wir vermuten, dass die Hausherrin sg sein Gast pl ein Wein sg angebietet hat., Wir denken, dass der Mann sg sein Freund pl eine Sehenswürdigkeit pl gezeigt hat., Wir bezweifeln, dass der Verdächtiger sg sein Anwalt sg die Wahrheit sg verheimlicht hat., Wir denken, dass der Arzt sg der Patient sg eine Behandlung sg verschieben hat., Wir erfahren, dass der Eigentümer sg ein Student sg seine Wohnung sg vermietet hat., Wir verdächtigen, dass der Spion sg ihr Feind sg das Geheimnis pl verratet hat., Wir denken, dass die Frau sg sein Partner sg eine Überraschung sg versprochen hat., Ihr weisst, dass die Mutter sg ihre Tochter sg ein Paket sg geschickt hat., Sie sagen, dass der Sportler sg seine Nachbarin sg sein Fahrrad sg verliehen hat., Wir erfahren, dass die Pr

In [80]:
tokens_w_number = [[] for _ in range(len(list_sentences_w_number))]
for idx_sentence, sentence in enumerate(doc_list_w_number):
    for token in sentence: 
        tokens_w_number[idx_sentence].append(token.text)

In [81]:
print(tokens_w_number)

[['Wir', 'glauben', ',', 'dass', 'der', 'Mann', 'sg', 'die', 'Frau', 'pl', 'ein', 'Geschenk', 'sg', 'gegeben', 'hat', '.'], ['Wir', 'wissen', ',', 'dass', 'der', 'Bankier', 'sg', 'sein', 'Kunde', 'pl', 'eine', 'Finanzierung', 'sg', 'angeboten', 'hat', '.'], ['Wir', 'wissen', ',', 'dass', 'das', 'Kindermädchen', 'sg', 'das', 'Kind', 'sg', 'ein', 'Buch', 'sg', 'gekauft', 'hat', '.'], ['Wir', 'vermuten', ',', 'dass', 'die', 'Hausherrin', 'sg', 'sein', 'Gast', 'pl', 'ein', 'Wein', 'sg', 'angebietet', 'hat', '.'], ['Wir', 'denken', ',', 'dass', 'der', 'Mann', 'sg', 'sein', 'Freund', 'pl', 'eine', 'Sehenswürdigkeit', 'pl', 'gezeigt', 'hat', '.'], ['Wir', 'bezweifeln', ',', 'dass', 'der', 'Verdächtiger', 'sg', 'sein', 'Anwalt', 'sg', 'die', 'Wahrheit', 'sg', 'verheimlicht', 'hat', '.'], ['Wir', 'denken', ',', 'dass', 'der', 'Arzt', 'sg', 'der', 'Patient', 'sg', 'eine', 'Behandlung', 'sg', 'verschieben', 'hat', '.'], ['Wir', 'erfahren', ',', 'dass', 'der', 'Eigentümer', 'sg', 'ein', 'Student',

In [82]:
doc_list_wo_number = []
for sentence in range(len(list_sentences_wo_number)):
    doc_list_wo_number.append(nlp(list_sentences_wo_number[sentence]))

In [83]:
print(doc_list_wo_number)

[Wir glauben, dass der Mann die Frau ein Geschenk gegeben hat., Wir wissen, dass der Bankier sein Kunde eine Finanzierung angeboten hat., Wir wissen, dass das Kindermädchen das Kind ein Buch gekauft hat., Wir vermuten, dass die Hausherrin sein Gast ein Wein angebietet hat., Wir denken, dass der Mann sein Freund eine Sehenswürdigkeit gezeigt hat., Wir bezweifeln, dass der Verdächtiger sein Anwalt die Wahrheit verheimlicht hat., Wir denken, dass der Arzt der Patient eine Behandlung verschieben hat., Wir erfahren, dass der Eigentümer ein Student seine Wohnung vermietet hat., Wir verdächtigen, dass der Spion ihr Feind das Geheimnis verratet hat., Wir denken, dass die Frau sein Partner eine Überraschung versprochen hat., Ihr weisst, dass die Mutter ihre Tochter ein Paket geschickt hat., Sie sagen, dass der Sportler seine Nachbarin sein Fahrrad verliehen hat., Wir erfahren, dass die Presse der Zuhörer die Nachricht verkündet hat., Sie hören, dass der Senat die Abgeordnete eine Maßnahme gemel

In [84]:
tokens_wo_number = [[] for _ in range(len(list_sentences_wo_number))]
for idx_sentence, sentence in enumerate(doc_list_wo_number):
    for token in sentence: 
        tokens_wo_number[idx_sentence].append(token.text)

In [85]:
print(tokens_w_number)

[['Wir', 'glauben', ',', 'dass', 'der', 'Mann', 'sg', 'die', 'Frau', 'pl', 'ein', 'Geschenk', 'sg', 'gegeben', 'hat', '.'], ['Wir', 'wissen', ',', 'dass', 'der', 'Bankier', 'sg', 'sein', 'Kunde', 'pl', 'eine', 'Finanzierung', 'sg', 'angeboten', 'hat', '.'], ['Wir', 'wissen', ',', 'dass', 'das', 'Kindermädchen', 'sg', 'das', 'Kind', 'sg', 'ein', 'Buch', 'sg', 'gekauft', 'hat', '.'], ['Wir', 'vermuten', ',', 'dass', 'die', 'Hausherrin', 'sg', 'sein', 'Gast', 'pl', 'ein', 'Wein', 'sg', 'angebietet', 'hat', '.'], ['Wir', 'denken', ',', 'dass', 'der', 'Mann', 'sg', 'sein', 'Freund', 'pl', 'eine', 'Sehenswürdigkeit', 'pl', 'gezeigt', 'hat', '.'], ['Wir', 'bezweifeln', ',', 'dass', 'der', 'Verdächtiger', 'sg', 'sein', 'Anwalt', 'sg', 'die', 'Wahrheit', 'sg', 'verheimlicht', 'hat', '.'], ['Wir', 'denken', ',', 'dass', 'der', 'Arzt', 'sg', 'der', 'Patient', 'sg', 'eine', 'Behandlung', 'sg', 'verschieben', 'hat', '.'], ['Wir', 'erfahren', ',', 'dass', 'der', 'Eigentümer', 'sg', 'ein', 'Student',

In [87]:
print(tokens_wo_number)

[['Wir', 'glauben', ',', 'dass', 'der', 'Mann', 'die', 'Frau', 'ein', 'Geschenk', 'gegeben', 'hat', '.'], ['Wir', 'wissen', ',', 'dass', 'der', 'Bankier', 'sein', 'Kunde', 'eine', 'Finanzierung', 'angeboten', 'hat', '.'], ['Wir', 'wissen', ',', 'dass', 'das', 'Kindermädchen', 'das', 'Kind', 'ein', 'Buch', 'gekauft', 'hat', '.'], ['Wir', 'vermuten', ',', 'dass', 'die', 'Hausherrin', 'sein', 'Gast', 'ein', 'Wein', 'angebietet', 'hat', '.'], ['Wir', 'denken', ',', 'dass', 'der', 'Mann', 'sein', 'Freund', 'eine', 'Sehenswürdigkeit', 'gezeigt', 'hat', '.'], ['Wir', 'bezweifeln', ',', 'dass', 'der', 'Verdächtiger', 'sein', 'Anwalt', 'die', 'Wahrheit', 'verheimlicht', 'hat', '.'], ['Wir', 'denken', ',', 'dass', 'der', 'Arzt', 'der', 'Patient', 'eine', 'Behandlung', 'verschieben', 'hat', '.'], ['Wir', 'erfahren', ',', 'dass', 'der', 'Eigentümer', 'ein', 'Student', 'seine', 'Wohnung', 'vermietet', 'hat', '.'], ['Wir', 'verdächtigen', ',', 'dass', 'der', 'Spion', 'ihr', 'Feind', 'das', 'Geheimni

In [88]:
print("sentences in tokens", len(tokens_w_number))
print("characters in tokens with number:", len(tokens_w_number[0]))
print("characters in tokens without number:", len(tokens_wo_number[0]))

sentences in tokens 19
characters in tokens with number: 16
characters in tokens without number: 13


### Selecting verb arguments

In [89]:
args_wo_number = [[] for _ in range(len(tokens_wo_number))]
for sentence_idx, sentence in enumerate(tokens_wo_number):
    args_wo_number[sentence_idx].append(tokens_wo_number[sentence_idx][4]+" "+tokens_wo_number[sentence_idx][5])
    args_wo_number[sentence_idx].append(tokens_wo_number[sentence_idx][6]+" "+tokens_wo_number[sentence_idx][7])
    args_wo_number[sentence_idx].append(tokens_wo_number[sentence_idx][8]+" "+tokens_wo_number[sentence_idx][9])
print(args_wo_number)

[['der Mann', 'die Frau', 'ein Geschenk'], ['der Bankier', 'sein Kunde', 'eine Finanzierung'], ['das Kindermädchen', 'das Kind', 'ein Buch'], ['die Hausherrin', 'sein Gast', 'ein Wein'], ['der Mann', 'sein Freund', 'eine Sehenswürdigkeit'], ['der Verdächtiger', 'sein Anwalt', 'die Wahrheit'], ['der Arzt', 'der Patient', 'eine Behandlung'], ['der Eigentümer', 'ein Student', 'seine Wohnung'], ['der Spion', 'ihr Feind', 'das Geheimnis'], ['die Frau', 'sein Partner', 'eine Überraschung'], ['die Mutter', 'ihre Tochter', 'ein Paket'], ['der Sportler', 'seine Nachbarin', 'sein Fahrrad'], ['die Presse', 'der Zuhörer', 'die Nachricht'], ['der Senat', 'die Abgeordnete', 'eine Maßnahme'], ['der Junge', 'sein Kumpel', 'ein Kugelschreiber'], ['das Theater', 'der Musiker', 'ein Sitzplatz'], ['der Schüler', 'der Lehrer', 'ihre Frage'], ['der Professor', 'der Student', 'ein Fach'], ['der Chef', 'das Kind', 'ein Kuchen']]


In [90]:
args_number = 3
args_w_number = [[[] for _ in range(args_number)] for _ in range(len(tokens_w_number))]
for sentence_idx, sentence in enumerate(tokens_w_number):
    args_w_number[sentence_idx][0]=(tokens_w_number[sentence_idx][4:7])
    args_w_number[sentence_idx][1]=(tokens_w_number[sentence_idx][7:10])
    args_w_number[sentence_idx][2]=(tokens_w_number[sentence_idx][10:13])
print(args_w_number)

[[['der', 'Mann', 'sg'], ['die', 'Frau', 'pl'], ['ein', 'Geschenk', 'sg']], [['der', 'Bankier', 'sg'], ['sein', 'Kunde', 'pl'], ['eine', 'Finanzierung', 'sg']], [['das', 'Kindermädchen', 'sg'], ['das', 'Kind', 'sg'], ['ein', 'Buch', 'sg']], [['die', 'Hausherrin', 'sg'], ['sein', 'Gast', 'pl'], ['ein', 'Wein', 'sg']], [['der', 'Mann', 'sg'], ['sein', 'Freund', 'pl'], ['eine', 'Sehenswürdigkeit', 'pl']], [['der', 'Verdächtiger', 'sg'], ['sein', 'Anwalt', 'sg'], ['die', 'Wahrheit', 'sg']], [['der', 'Arzt', 'sg'], ['der', 'Patient', 'sg'], ['eine', 'Behandlung', 'sg']], [['der', 'Eigentümer', 'sg'], ['ein', 'Student', 'sg'], ['seine', 'Wohnung', 'sg']], [['der', 'Spion', 'sg'], ['ihr', 'Feind', 'sg'], ['das', 'Geheimnis', 'pl']], [['die', 'Frau', 'sg'], ['sein', 'Partner', 'sg'], ['eine', 'Überraschung', 'sg']], [['die', 'Mutter', 'sg'], ['ihre', 'Tochter', 'sg'], ['ein', 'Paket', 'sg']], [['der', 'Sportler', 'sg'], ['seine', 'Nachbarin', 'sg'], ['sein', 'Fahrrad', 'sg']], [['die', 'Presse

In [91]:
print(len(args_w_number))
print(len(args_w_number[0]))

19
3


### Permuting verb argument positions

In [92]:
# permute noun phrases
from itertools import permutations
shuffled_args = [[] for _ in range(len(args_wo_number))]
for i in range(len(args_wo_number)):
    shuffled_args[i] = list(permutations(args_wo_number[i]))
print(shuffled_args[0])

[('der Mann', 'die Frau', 'ein Geschenk'), ('der Mann', 'ein Geschenk', 'die Frau'), ('die Frau', 'der Mann', 'ein Geschenk'), ('die Frau', 'ein Geschenk', 'der Mann'), ('ein Geschenk', 'der Mann', 'die Frau'), ('ein Geschenk', 'die Frau', 'der Mann')]


In [100]:
# shuffle positions
shuffled_sentences = [[] for _ in range(len(args_wo_number))]
for sentence_idx, sentence in enumerate(tokens_wo_number): 
    for noun_group in shuffled_args[sentence_idx]:
        principal_clause = f"{tokens_wo_number[sentence_idx][0]} {tokens_wo_number[sentence_idx][1]},{tokens_wo_number[sentence_idx][3]}"
        verb_arguments = f"{noun_group[0]} {noun_group[1]} {noun_group[2]}"
        verb_subordinate = f"{tokens_wo_number[sentence_idx][-3]} {tokens_wo_number[sentence_idx][-2]}"
        shuffled_sentences[sentence_idx].append(f"{principal_clause} {verb_arguments} {verb_subordinate}.")
        
print("original sentences:", len(shuffled_sentences))
print("generated sentences per original sentence:",len(shuffled_sentences[0]))
print(shuffled_sentences)


original sentences: 19
generated sentences per original sentence: 6
[['Wir glauben,dass der Mann die Frau ein Geschenk gegeben hat.', 'Wir glauben,dass der Mann ein Geschenk die Frau gegeben hat.', 'Wir glauben,dass die Frau der Mann ein Geschenk gegeben hat.', 'Wir glauben,dass die Frau ein Geschenk der Mann gegeben hat.', 'Wir glauben,dass ein Geschenk der Mann die Frau gegeben hat.', 'Wir glauben,dass ein Geschenk die Frau der Mann gegeben hat.'], ['Wir wissen,dass der Bankier sein Kunde eine Finanzierung angeboten hat.', 'Wir wissen,dass der Bankier eine Finanzierung sein Kunde angeboten hat.', 'Wir wissen,dass sein Kunde der Bankier eine Finanzierung angeboten hat.', 'Wir wissen,dass sein Kunde eine Finanzierung der Bankier angeboten hat.', 'Wir wissen,dass eine Finanzierung der Bankier sein Kunde angeboten hat.', 'Wir wissen,dass eine Finanzierung sein Kunde der Bankier angeboten hat.'], ['Wir wissen,dass das Kindermädchen das Kind ein Buch gekauft hat.', 'Wir wissen,dass das Kin

### Generate case permutations

In [141]:
def get_nom(group_args):
    # get nominative of verb argument and corresponding article
    new_nom = []
    article = group_args[0]
    word = group_args[1]
    number = group_args[2]
    
    # get singular 
    if number == "sg":
        new_nom.append(article+" "+word)
    
    # get plural
    else:
        pl_article = ""
        pl_word = ""
        if article in {"der","die","das"}:
            pl_article = "die"
        elif article in {"sein","seine"}:
            pl_article = "seine"
        elif article in {"ihr","ihre"}:
            pl_article = "ihre"
        elif article in {"ein","eine"}:
            pl_article = "einige"
        
        pl_word = german_dict[word][4]
        
        new_nom.append(pl_article+" "+pl_word)
        
    return new_nom

In [142]:
print(get_nom(args_w_number[0][0]))
print(get_nom(args_w_number[0][1]))
print(get_nom(args_w_number[17][2]))
print(get_nom(args_w_number[18][2]))

['der Mann']
['die Frauen']
['einige Fächer']
['ein Kuchen']


In [143]:
def get_acc(group_args):
    # get accusative of verb argument and corresponding article
    new_acc = []
    article = group_args[0]
    word = group_args[1]
    gender = german_dict[word][0]
    number = group_args[2]
    
    # singular
    if number == "sg":
        sg_article = ""
        sg_word = ""
        if gender == "m":
            if article == "der":
                sg_article = "den"
            elif article == "sein":
                sg_article = "seinen"
            elif article == "ihr":
                sg_article = "ihren"
            else:
                sg_article = "einen"
        else:
            sg_article = article
        
        sg_word = german_dict[word][1]
        
        new_acc.append(sg_article+" "+sg_word)
                
    # plural        
    else: 
        pl_article = ""
        pl_word = ""
        if article in {"der","die","das"}:
            pl_article = "die"
        elif article in {"sein","seine"}:
            pl_article = "seine"
        elif article in {"ihr","ihre"}:
            pl_article = "ihre"
        elif article in {"ein","eine"}:
            pl_article = "einige"
        #elif article in {"ein","eine"}:
            
    
        pl_word = german_dict[word][5]   
        
        new_acc.append(pl_article+" "+pl_word)
        
    return new_acc

In [144]:
print(get_acc(args_w_number[0][0]))
print(get_acc(args_w_number[0][1]))
print(get_acc(args_w_number[1][0]))
print(get_acc(args_w_number[17][2]))

['den Mann']
['die Frauen']
['den Bankier']
['einige Fächer']


In [149]:
def get_dat(group_args):
    # get dative of verb argument and corresponding article
    new_dat = []
    article = group_args[0]
    word = group_args[1]
    gender = german_dict[word][0]
    number = group_args[2]
    
    if number == "sg":
        sg_article = ""
        sg_word = ""
        if gender in {"m","n"}:
            if article in {"der","das"}:
                sg_article = "dem"
            elif article == "sein":
                sg_article = "seinem"
            elif article == "ihr":
                sg_article = "ihrem"
            elif article == "ein":
                sg_article = "einem"
        elif gender == "f":
            if article == "die":
                sg_article = "der"
            elif article == "seine":
                sg_article = "seiner"
            elif article == "ihre":
                sg_article = "ihrer"
            elif article == "eine":
                sg_article = "einer"
        
        sg_word = german_dict[word][2] 
        new_dat.append(sg_article+" "+sg_word)
        
    else: 
        pl_article = ""
        pl_word = ""
        if article in {"der","die","das"}:
            pl_article = "den"
        elif article in {"sein","seine"}:
            pl_article = "seinen"
        elif article in {"ihr","ihre"}:
            pl_article = "ihren" 
        elif article in {"ein","eine"}:
            pl_article = "einigen"
    
        pl_word = german_dict[word][6]   
        
        new_dat.append(pl_article+" "+pl_word)
        
    return new_dat

In [150]:
print(get_dat(args_w_number[0][0]))
print(get_dat(args_w_number[0][1]))
print(get_dat(args_w_number[17][0]))
print(get_dat(args_w_number[17][1]))
print(get_dat(args_w_number[17][2]))
print(get_nom(args_w_number[18][2]))

['dem Mann']
['den Frauen']
['dem Professor']
['den Studenten']
['einigen Fächern']
['ein Kuchen']


In [151]:
def get_all_cases(list_args, number_args):
    all_cases = [[[] for _ in range(number_args)] for _ in range(len(args_w_number))]
    for sentence_idx, sentence in enumerate(args_w_number):
        for group_args_idx, group_args in enumerate(args_w_number[sentence_idx]):
            nom = get_nom(args_w_number[sentence_idx][group_args_idx])
            acc = get_acc(args_w_number[sentence_idx][group_args_idx])
            dat = get_dat(args_w_number[sentence_idx][group_args_idx])
            all_cases[sentence_idx][group_args_idx].append(nom)
            all_cases[sentence_idx][group_args_idx].append(acc)
            all_cases[sentence_idx][group_args_idx].append(dat)
    return all_cases

In [152]:
get_all_cases(args_w_number, 3)

[[[['der Mann'], ['den Mann'], ['dem Mann']],
  [['die Frauen'], ['die Frauen'], ['den Frauen']],
  [['ein Geschenk'], ['ein Geschenk'], ['einem Geschenk']]],
 [[['der Bankier'], ['den Bankier'], ['dem Bankier']],
  [['seine Kunden'], ['seine Kunden'], ['seinen Kunden']],
  [['eine Finanzierung'], ['eine Finanzierung'], ['einer Finanzierung']]],
 [[['das Kindermädchen'], ['das Kindermädchen'], ['dem Kindermädchen']],
  [['das Kind'], ['das Kind'], ['dem Kind']],
  [['ein Buch'], ['ein Buch'], ['einem Buch']]],
 [[['die Hausherrin'], ['die Hausherrin'], ['der Hausherrin']],
  [['seine Gasten'], ['seine Gasten'], ['seinen Gasten']],
  [['ein Wein'], ['einen Wein'], ['einem Wein']]],
 [[['der Mann'], ['den Mann'], ['dem Mann']],
  [['seine Freunde'], ['seine Freunde'], ['seinen Freunden']],
  [['einige Sehenswürdigkeiten'],
   ['einige Sehenswürdigkeiten'],
   ['einigen Sehenswürdigkeiten']]],
 [[['der Verdächtiger'], ['den Verdächtigen'], ['dem Verdächtigem']],
  [['sein Anwalt'], ['sein